In [1]:
pip install xml.etree.ElementTree

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement xml.etree.ElementTree (from versions: none)
ERROR: No matching distribution found for xml.etree.ElementTree

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## DICOM Standard harvesting of Part 3 
### Extract relationships between tags and CIDs

In [1]:
import requests
import xml.etree.ElementTree as ET

# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml'

# Parse the XML content
response = requests.get(xml_uri)
root = ET.fromstring(response.content)

In [16]:
for child in root:
    print(child.tag)


{http://docbook.org/ns/docbook}title
{http://docbook.org/ns/docbook}subtitle
{http://docbook.org/ns/docbook}info
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docbook}chapter
{http://docbook.org/ns/docboo

In [17]:
import pandas as pd
from collections import defaultdict

# Get table 1 from Chapter 6
if root is not None:
    grandchild_table = selected_node.find('.//{http://docbook.org/ns/docbook}table')
    if grandchild_table is not None:
        thead = grandchild_table.find('.//{http://docbook.org/ns/docbook}thead')
        tbody = grandchild_table.find('.//{http://docbook.org/ns/docbook}tbody')

        # Store column names
        column_names = []
        if thead is not None:
            for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                thead_names = tr.findall('.//{http://docbook.org/ns/docbook}emphasis')
                if thead_names is not None:
                    for name in thead_names:
                        column_names.append(name.text.strip())

            # Store row values
            rows = tbody.findall('.//{http://docbook.org/ns/docbook}tr')
            rows_data = []

            # Loop through tbody to extract values
            for tr in rows:
                row_values = defaultdict(lambda: None)
                idx = 0
                for para in tr.findall('.//{http://docbook.org/ns/docbook}para'):
                    emphasis = para.find('.//{http://docbook.org/ns/docbook}emphasis')
                    if emphasis is not None and emphasis.text is not None:
                        row_values[column_names[idx]] = emphasis.text.strip()
                        idx += 1
                        if idx >= len(column_names):
                            break
                    else:
                        if para is not None and para.text is not None:
                            row_values[column_names[idx]] = para.text.strip()
                            idx += 1
                            if idx >= len(column_names):
                                break

                # Append to the table only if it has more than 3 values
                if sum(1 for value in row_values.values() if value is not None) > 3:
                    rows_data.append(row_values)

            # Save the output as a DataFrame
            df = pd.DataFrame(rows_data, columns=column_names)
            print(df)
else:
    print("Node with label='6' not found.")

NameError: name 'selected_node' is not defined